In [84]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Apr  9 15:41:43 2020.

@author: tagir.farkhutdinov@atco.com
"""

import sympy
from IPython.display import display, Latex

In [2]:
"""Print equations."""
# pylint: disable=C0103, R0914
_sp = sy = sympy
sympify = sympy.sympify
x, t, rho_f, rho_s0, g_0, K = _sp.symbols('x t rho_f rho_s^0 g_0 K')
X, Y, P = _sp.symbols('X Y P', cls=_sp.Function)

X = X(x, t)
Y = Y(x, t)

u_s, u_f = _sp.symbols('u_s u_f', cls=_sp.Function)
u_s, u_f = u_s(x, t), u_f(x, t)
g, rho_s = _sp.symbols('g rho_s')

def lhs(u, denom):
    return ((u.diff(t) + u*u.diff(x)) * denom.diff(x)
            ).simplify().factor(fraction=False)

def simplify_eqs(eq):
    return _sp.Eq((-eq.lhs).expand().factor(fraction=False),
                  -eq.rhs)

p_dx_expr = P(X.diff(x), Y.diff(x)).diff(x)
p_dx = _sp.Symbol("\\partial_{x} P")
sigma_s_dx = _sp.Symbol("\\partial_{x}(\\sigma_e + S)")

def eq_builder(u_f, u_s, g, rho_s, rho_f):
    eq_fluid = _sp.Eq(rho_f*g*lhs(u_f, x),
                      (-g*sympify('(p(x,t) + mu(x, t))').diff(x) + K*(u_s - u_f)))
    eq_solid = _sp.Eq(rho_s*lhs(u_s, x),
                      (+g*sympify('p(x,t)').diff(x) - (1-g)*sympify('mu(x,t)').diff(x) + sigma_s_dx + K*(u_f - u_s)))
    eq_p_dx = _sp.Eq(-p_dx, -p_dx_expr)
    return eq_fluid, eq_solid, eq_p_dx

def subscr_partials(eq, before, after):
    return eq.subs(sympify(before).diff(t, t), sympify(after + '_tt')
                  ).subs(sympify(before).diff(x, x), sympify(after + '_xx')
                        ).subs(sympify(before).diff(x, t), sympify(after + '_xt')
                              ).subs(sympify(before).diff(x), sympify(after + '_x')
                                    ).subs(sympify(before).diff(t), sympify(after + '_t')
                                          )
                                     
def nice_partials(eq):
    return subscr_partials(subscr_partials(eq, 'Y(x, t)', 'Y'), 'X(x, t)', 'X')

def print_eqn(eq, oneline=True, nicepartials=True):
    # pylint: disable=W0212
    eq = nice_partials(eq)
    def correct_render(latex_repr):
        return latex_repr.replace(
            '\\substack', '').replace(
                '\\displaystyle', '').replace(
                    r'\xi_{1}', r'\xi').replace('\\xi_{2}', '\\eta')
    if not oneline:
        display(Latex(correct_render(eq.lhs._repr_latex_())))
        display(Latex('$\\quad=' +
                      correct_render(eq.rhs._repr_latex_()[1:-1])
                      + ',$'))
    else:
        display(Latex(correct_render(eq._repr_latex_())))


In [3]:
# Original equations
eq_fluid, eq_solid, eq_p_dx = eq_builder(u_f, u_s, g, rho_s, rho_f)
print('Fluid dynamics:')
print_eqn(eq_fluid, True)
print('Solid dynamics:')
print_eqn(eq_solid, True)
# Substitutions of X and Y
u_s_ = -X.diff(t)/X.diff(x)
u_f_ = -Y.diff(t)/Y.diff(x)
g_ = g_0 * Y.diff(x)
rho_s_ = rho_s0 * X.diff(x)
eq_fluid, eq_solid, eq_p_dx = eq_builder(u_f_, u_s_, g_, rho_s_, rho_f)

eq_fluid, eq_solid, eq_p_dx = list(
    map(simplify_eqs, [eq_fluid, eq_solid, eq_p_dx]))

print('Dynamics of fluid and solid after substitution:')
print_eqn(eq_fluid, True)
print_eqn(eq_solid, True)
print('Pressure:')
print_eqn(eq_p_dx, True)

Fluid dynamics:


<IPython.core.display.Latex object>

Solid dynamics:


<IPython.core.display.Latex object>

Dynamics of fluid and solid after substitution:


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

Pressure:


<IPython.core.display.Latex object>

In [4]:
print("Total incompressibility:")
print("DON'T WORRY about missing braces after partials x,\nit's solely a sympy representation bug):")

def eq_incompr(g, u_f, u_s): return sy.Eq(sy.Derivative(g*u_f, x),
                                          -sy.Derivative(((1-g)*u_s), x))
print_eqn(eq_incompr(g, u_f, u_s), True)

print("After substitutions:")
print_eqn(eq_incompr(g_, u_f_, u_s_), True)
print_eqn(eq_incompr(g_, u_f_, u_s_).doit(), True)
print("Multipled by X_x**2:")
print_eqn(sy.Eq((X.diff(x)**2 * 
                (eq_incompr(g_, u_f_, u_s_).doit().lhs + g_0*X.diff(t)*Y.diff(x, x)/X.diff(x))),
                (X.diff(x)**2 * 
                 (eq_incompr(g_, u_f_, u_s_).doit().rhs + g_0*X.diff(t)*Y.diff(x, x)/X.diff(x))
                )
               ).simplify()
         )

print("In our paper")
print_eqn(sy.Eq(g_0 * X.diff(x) * Y.diff(t) + (1 - g_0 * Y.diff(x)) * X.diff(t), 0))
print("- Partial wrt x")
print_eqn(sy.Eq(-(g_0 * X.diff(x) * Y.diff(t) + (1 - g_0 * Y.diff(x)) * X.diff(t)).diff(x), 0))

print("It seems that it should be ")
eq_incompr_correct = sy.Eq(
    -g_0 * sy.Derivative(Y.diff(t)/X.diff(x), x) * X.diff(x)**3,
    (1 - g_)*sy.Derivative(X.diff(t)/X.diff(x), x) * X.diff(x)**2
)
print_eqn(eq_incompr_correct)
print("Partial wrt x")
print_eqn(sy.Eq((eq_incompr_correct.lhs.doit()).simplify(),
                (eq_incompr_correct.rhs.doit()).simplify())
         )

Total incompressibility:
DON'T WORRY about missing braces after partials x,
it's solely a sympy representation bug):


<IPython.core.display.Latex object>

After substitutions:


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

It seems that it should be 


<IPython.core.display.Latex object>

Partial wrt x


<IPython.core.display.Latex object>

In [5]:
# Total incompressibility w.r.t. Time
def incompr_t(g, u_f, u_s): return sy.Eq(sy.Derivative(g*u_f, t),
                                          -sy.Derivative(((1-g)*u_s), t))

print_eqn(incompr_t(g, u_f, u_s), True)

print("After substitutions:")
print_eqn(incompr_t(g_, u_f_, u_s_), True)
print_eqn(incompr_t(g_, u_f_, u_s_).doit(), True)

print("We introduce")
mu = sympify('mu(x, t)')
X_tt = sy.Eq(sympify('X_tt'),
             sympify('R_X') + (1 - g_0*Y.diff(x))/rho_s0 * mu.diff(x))
Y_tt = sy.Eq(sympify('Y_tt'),
             sympify('R_Y') + Y.diff(x)/rho_f * mu.diff(x))
print_eqn(X_tt)
print_eqn(Y_tt)

print("and substitute into the time derivative of incomressibility")
incompr_t_done = incompr_t(g_, u_f_, u_s_
                          ).doit().subs(Y.diff(t, t), Y_tt.rhs
                                       ).subs(X.diff(t, t), X_tt.rhs
                                             ).subs(mu.diff(x), sympify('mu_x')
                                                   )

print_eqn(incompr_t_done)
mu_eq = nice_partials(incompr_t_done.lhs - incompr_t_done.rhs
                      ).expand().collect(sympify('mu_x')
                                        ) * sympify('X_x')
mu_eq = mu_eq.expand()
print("Collect terms with mu_x")
# print_eqn(mu_eq)
mu_coef = mu_eq.coeff(sympify('mu_x'), 1).simplify()
mu_eq = sy.Eq(mu_coef*sympify('mu_x'),
              (mu_coef*sympify('mu_x') - mu_eq).simplify().collect(sympify('R_X')))
print_eqn(mu_eq)

<IPython.core.display.Latex object>

After substitutions:


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

We introduce


...

<IPython.core.display.Latex object>

and substitute into the time derivative of incomressibility


<IPython.core.display.Latex object>

Collect terms with mu_x


<IPython.core.display.Latex object>

In [67]:
rho_s = X.diff(x)
V = sy.symbols('V', cls=sy.Function)
b, v = sy.symbols('b v')

def potential_eqn(rho_s, rho_f):
    V = sy.symbols('V', cls=sy.Function)
    return sy.Eq(V(rho_s, rho_f),
        sy.Symbol('alpha')/2 * (rho_s - 1)**2
        + sy.Symbol('beta')/2 * (rho_f*g_0 - (1-(1-g_0)*rho_s))**2
        + sy.Symbol('gamma')/2 * (1 - rho_f*g_0) * (rho_s - 1)**2 
    )

print('Potential energy definition and transformations:')
eq_potential = potential_eqn(X.diff(x), g/g_0)
print_eqn(eq_potential)

eq_potential = eq_potential.subs(X.diff(x), 1/_sp.sqrt(b))
print_eqn(eq_potential)

eq_potential = eq_potential.subs(sympify('g'), sympify('g_0*c_0')*v / sy.sqrt(b))
print_eqn(eq_potential)

Potential energy definition and transformations:


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [82]:
print('Sigma epsilon:')
b, v = sympify('b, v')
V = sympify('V(v, b)')
sigma_e = sy.Eq(sympify('sigma_e'), V + 2*sy.Derivative(V, b)*b)
print_eqn(sigma_e)

sigma_e = sigma_e.subs(V, eq_potential.rhs).doit()
print_eqn(sigma_e)

print('Back substitutions of X and Y')
sigma_e = sigma_e.subs(sy.sqrt(b), 1/X.diff(x)
                      ).subs(v, sympify('g_0/c_0')*Y.diff(x)/X.diff(x)
                            ).simplify()


def collect(expr, symbols):
    result = sympify(0)
    for symbol in symbols:
        coeff = sy.collect(expr.expand(), symbol).coeff(symbol, 1).factor()
        result += symbol*coeff
    return result

def collect_eq(eq, symbols):
    result = sympify(0)
    rhs = eq.rhs.expand()
    for symbol in symbols:
        coeff = sy.collect(rhs, symbol).coeff(symbol, 1).factor()
        result += symbol*coeff
    return sy.Eq(eq.lhs, result)

sigma_e = collect_eq(sigma_e, sy.symbols('alpha beta gamma'))
print_eqn(sigma_e)

print('We used the following formula for the pore volume in derivation above')
pore_vol_eq = sy.Eq(v, sympify('g_0/c_0')*Y.diff(x)/X.diff(x))
print_eqn(pore_vol_eq)

print('Derivative of sigma_e w.r.t. x')
sigma_e_x = sy.Eq(sy.Derivative(sympify('sigma_e(x, t)'), x),
                  sigma_e.rhs.diff(x).simplify())
print_eqn(collect_eq(sigma_e_x, sy.symbols('alpha beta gamma')))

Sigma epsilon:


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

Back substitutions of X and Y


<IPython.core.display.Latex object>

We used the following formula for the pore volume in derivation above


<IPython.core.display.Latex object>

Derivative of sigma_e w.r.t. x


<IPython.core.display.Latex object>

In [83]:
print("Finally, let's compute the pressure, using the potential")
pressure_eq = _sp.Eq(sympify('p*c(b)'), V.diff(v))

print_eqn(pressure_eq)
pressure_eq = pressure_eq.subs(V, eq_potential.rhs).doit()
print_eqn(pressure_eq)

print("That gives us")
pressure_eq = sy.Eq(sympify('p'), pressure_eq.rhs / (sympify('c_0') / sy.sqrt(b))).simplify()
print_eqn(pressure_eq)

print('Back substitutions of X and Y')
pressure_eq = pressure_eq.subs(sy.sqrt(b), 1/X.diff(x)
                              ).subs(v, sympify('g_0/c_0')*Y.diff(x)/X.diff(x)
                                    ).simplify()
print_eqn(pressure_eq)

print('Derivative of pressure w.r.t. x')
pressure_x = sy.Eq(sy.Derivative(sympify('p(x, t)'), x), pressure_eq.rhs.diff(x).simplify())
print_eqn(pressure_x)

Finally, let's compute the pressure, using the potential


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

That gives us


<IPython.core.display.Latex object>

Back substitutions of X and Y


<IPython.core.display.Latex object>

Derivative of pressure w.r.t. x


<IPython.core.display.Latex object>